# One Score to Rule Them All
The time finally comes to assess which neighborhood might be worthy of the title of best! We decided to give each neighborhood a score out of 100 in order to simplify the process.

Each member of the team decided on an equation to give the neighborhoods a score, based on their individual metric.

In [1]:
import pandas as pd
import geopandas as gp
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
#EBLL score
nb = gp.read_file("neighborhoods/Neighborhoods_.shp")
ell = pd.read_csv("elevated-blood-lead-levels.csv")
nb.insert(0,"tract",nb['tractce10'].fillna(0).astype("int64", errors='ignore'))
#nb.insert(0,"tract",nb['tractce10'].astype("Int64", errors='ignore').fillna(0))
ell.insert(0,"tract",(ell["CensusTract"]-42003000000))
ellnb = nb.merge(ell, how="left", on="tract")
scored = ellnb[['hood','percentEBLL15_20']].dropna().sort_values('percentEBLL15_20')
scored.insert(2,'EBLLscore',100-ellnb['percentEBLL15_20']*10)
EBLLscored = scored[['hood','EBLLscore']]

In [3]:
#Education score
school_data = pd.read_csv('school-enrollment-by-detailed-level-of-school-for-the-population-3-years-and-over.csv')#open the file to read
percent_list = []
neigh_list = []
score_list = []

for index, row in school_data.iterrows():# go through each row 
    percent_list.append(100 * (row[4]/row[2]))     # puts percent into percent list  
    neigh_list.append(row[0])     # put neighborhood in
    
d = {'Neighborhood': neigh_list, 'Education Score of Neighborhood': percent_list}#create new data set with neighborhoods and percentages
percent_list = pd.DataFrame(data=d)# creates a new df so we can get the right scale for the points

for index, row in percent_list.iterrows():# go through each row 
    score_list.append(100 * (row[1]/percent_list["Education Score of Neighborhood"].max()))# append new score scale to score_list 

d = {'Neighborhood': neigh_list, 'Education Score of Neighborhood': score_list}#create new data set with neighborhoods and percentages
eduscored = pd.DataFrame(data=d)

In [4]:
police_arrest = pd.read_csv("police_arrest.csv")
filter1 = police_arrest["OFFENSES"].str.contains("dui", case = False, na = False)
filter2 = police_arrest["OFFENSES"].str.contains("assault", case = False, na = False)
p = police_arrest[filter1 | filter2]
percent = (p["INCIDENTNEIGHBORHOOD"].value_counts()/police_arrest["INCIDENTNEIGHBORHOOD"].value_counts()) * 100
percent.fillna(0, inplace = True)
score = 100 - percent
lawscored = score.sort_values()
ls = lawscored.to_frame().reset_index()

In [5]:
merged = EBLLscored.merge(eduscored, how='outer', left_on='hood', right_on='Neighborhood')
merged = merged.merge(ls, how='outer', left_on='hood', right_on='index').fillna(0)
merged.insert(0,'finalscore',merged['EBLLscore']*.4+merged['Education Score of Neighborhood']*.2+merged['INCIDENTNEIGHBORHOOD']*.4)
merged[['hood','finalscore']].sort_values('finalscore',ascending=False).head(10)

,hood,finalscore
2,West Oakland,76.865913
6,South Side Flats,75.310536
13,Central Business District,75.262341
1,Bloomfield,74.946938
8,East Liberty,74.626083
17,Squirrel Hill North,74.345770
11,Terrace Village,69.727802
9,Stanton Heights,67.986369
25,Shadyside,67.771504
16,Squirrel Hill South,67.451310


<p>Above are the top 10 safest neighborhoods to raise children based on the metrics all 3 of us used. We weighted each submetric with lead levels with 40%, Education Score with 20%, and assault and DUI cases with 40%. After calculating each weighted score, the top 10 neighborhoods we got are listed above with West Oakland in 1st and Squirrel Hill South in 10th.</p>